In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid,KFold

In [24]:
def feature_importance(data,features,param,inner_cv,outer_cv,Classifier,parameter_grid):
    
    data=data.sample(frac=1)
    X,y=np.array(data[features]), np.array(data[param])
    
    outer_scores=[]
    df_all= {}
    i=0
    
    for training_samples,test_samples in outer_cv.split(X,y):
        best_parms={}
        best_score=-np.inf
        i+=1
        for parameters in parameter_grid:
            cv_scores=[]
    
            for inner_train,inner_test in inner_cv.split(X[training_samples],y[training_samples]):
                clf=Classifier(**parameters)
                clf.fit(X[inner_train],y[inner_train])
                score=clf.score(X[inner_test],y[inner_test])
                cv_scores.append(score)
                
            mean_score=np.mean(cv_scores)
            
            if mean_score>best_score:
                best_score=mean_score
                best_params=parameters
                
        clf=Classifier(**best_params)
        clf.fit(X[training_samples],y[training_samples])
        outer_scores.append(clf.score(X[test_samples],y[test_samples]))
        
        
        ipdict = {n:s for n, s in zip(features, clf.feature_importances_)} 
        ser = pd.Series(ipdict) 
        df_all[i]=ser
        
    results=pd.concat([df_all[key] for key in df_all.keys()],axis=1,sort=False)
    mean=results.mean(axis='columns')
    mean=pd.Series(mean,name="mean")
    results=pd.concat([results,mean],axis=1)
    results=results.sort_values('mean', ascending=False).T
    
    b=[]
    for a in range(1,i+1):
        a=str(a)
        b.append(a)
    
    b.append("mean")
    results.index=b
    
    results.to_csv("feature_importance.csv")
    
    return results
    

In [ ]:
param_grid = {"learning_rate":[i/1000 for i in range(1,502,5)],  
                   "max_depth":[2,3,4,5],
                   "n_estimators":[300]}

In [8]:
data=pd.read_excel("Data.xlsx", sheet_name='regular')#Select regular or time-saving
data

,Unnamed: 0,L-Arginine hydrochloride,L-Glutamine,L-Histidine hydrochloride-H2O,L-Isoleucine,L-Leucine,L-Lysine hydrochloride,L-Methionine,L-Phenylalanine,L-Threonine,...,CaCl2,MgSO4,KCl,NaHCO3,NaCl,NaH2PO4,D-Glucose,FBS,mean_A450,sd_A450
0,1,0.6,2.0,0.2,0.4,0.4,0.4,0.1,0.20,0.4,...,1.8,0.8,5.5,26.0,120.0,1.0,5.6,0.10,1.309333,0.197250
1,2,0.6,2.0,0.2,0.4,0.4,0.4,0.1,0.20,0.4,...,1.8,0.8,5.5,26.0,120.0,1.0,5.6,0.10,0.029000,0.005437
2,3,0.6,2.0,0.2,0.4,0.4,0.4,0.1,0.20,0.4,...,1.8,0.8,5.5,26.0,120.0,1.0,5.6,0.10,1.519333,0.057285
3,4,0.6,2.0,0.2,0.4,0.4,0.4,0.1,0.20,0.4,...,1.8,0.8,5.5,26.0,120.0,1.0,5.6,0.10,1.152667,0.502906
4,5,0.6,2.0,0.2,0.4,0.4,0.4,0.1,0.20,0.4,...,1.8,0.8,5.5,26.0,120.0,1.0,5.6,0.10,1.584667,0.089432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,304,0.6,2.0,0.2,0.4,0.4,0.4,0.1,0.02,0.4,...,1.8,0.8,5.5,26.0,120.0,1.0,5.6,0.01,0.969556,0.030620
304,305,0.6,2.0,0.2,4.0,0.4,0.4,0.1,0.20,0.4,...,1.8,0.8,5.5,26.0,120.0,1.0,5.6,0.01,0.972556,0.123032
305,306,0.6,2.0,0.2,4.0,0.4,0.4,0.1,0.20,0.4,...,1.8,0.8,5.5,26.0,120.0,1.0,5.6,0.01,0.859556,0.055913
306,307,0.6,2.0,0.2,0.4,0.4,0.4,0.1,0.20,0.4,...,1.8,0.8,5.5,26.0,120.0,1.0,5.6,0.01,0.606222,0.013098


In [12]:
features=data.columns[1:30]
features

Index(['L-Arginine hydrochloride ', 'L-Glutamine ',
       'L-Histidine hydrochloride-H2O', 'L-Isoleucine ', 'L-Leucine ',
       'L-Lysine hydrochloride ', 'L-Methionine ', 'L-Phenylalanine ',
       'L-Threonine ', 'L-Tryptophan', 'L-Tyrosine disodium salt dihydrate',
       'L-Valine', 'L-Cystine 2HCl ', 'Choline chloride',
       'D-Calcium pantothenate ', 'Folic Acid', 'Niacinamide',
       'Pyridoxal hydrochloride', 'Riboflavin', 'Thiamine hydrochloride ',
       'i-Inositol', 'CaCl2', 'MgSO4', 'KCl ', 'NaHCO3', 'NaCl', 'NaH2PO4',
       'D-Glucose', 'FBS '],
      dtype='object')

In [ ]:
feature_importance(data,features,"mean_A450",KFold(5,shuffle=True),KFold(5,shuffle=True),GradientBoostingRegressor,ParameterGrid(param_grid))